# Jerusalem RAG Explorer - Retrieval and Question Answering

**By Yotam Nachtomy-Katz** | ID: 211718366 | Haifa University

This notebook demonstrates how to query the RAG system: semantic retrieval, context assembly, and answer generation with citations.

## 1. Setup and Imports

In [ ]:
import os
import json
from pathlib import Path

import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env")

# Check for API key
if not os.getenv("GEMINI_API_KEY"):
    print("WARNING: GEMINI_API_KEY not found. Answer generation will fail.")
else:
    print("GEMINI_API_KEY loaded successfully.")

## 2. Load Index and Model

In [ ]:
# Paths
INDEX_DIR = Path("../data/index_v2")
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# Language mappings
LANGUAGE_NAMES = {
    "en": "English",
    "la": "Latin",
    "ar": "Arabic",
    "el": "Greek",
    "fr": "French",
    "hy": "Armenian",
}

LANGUAGE_FLAGS = {
    "en": "🇬🇧",
    "la": "🇻🇦",
    "ar": "🇸🇦",
    "el": "🇬🇷",
    "fr": "🇫🇷",
    "hy": "🇦🇲",
}

def get_language_name(code: str) -> str:
    return LANGUAGE_NAMES.get(code, code.upper())

def get_language_flag(code: str) -> str:
    return LANGUAGE_FLAGS.get(code, "🌐")

In [ ]:
# Load embedding model
print(f"Loading embedding model: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME)

# Load FAISS index
index_path = INDEX_DIR / "faiss.index"
print(f"Loading FAISS index from {index_path}")
index = faiss.read_index(str(index_path))
print(f"Index contains {index.ntotal} vectors")

# Load chunks metadata
chunks_path = INDEX_DIR / "chunks.json"
print(f"Loading chunks from {chunks_path}")
with open(chunks_path, "r", encoding="utf-8") as f:
    chunks = json.load(f)
print(f"Loaded {len(chunks)} chunks")

## 3. Retrieval Function

The retrieval function embeds the query and searches for the most similar chunks in the FAISS index.

In [ ]:
def retrieve(
    question: str,
    top_k: int = 6,
    languages: list[str] = None,
) -> list[tuple[float, dict]]:
    """
    Retrieve top-k relevant chunks for a question.
    
    Args:
        question: Query string
        top_k: Number of results to return
        languages: Optional list of language codes to filter by
        
    Returns:
        List of (score, chunk_dict) tuples
    """
    # Encode query
    q_emb = model.encode([question], normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype="float32")

    # Search - get more if filtering
    search_k = top_k * 3 if languages else top_k
    scores, ids = index.search(q_emb, search_k)

    results = []
    for score, idx in zip(scores[0], ids[0]):
        if idx < 0 or idx >= len(chunks):
            continue

        chunk = chunks[idx]

        # Apply language filter
        if languages:
            chunk_lang = chunk.get("language", "en")
            orig_lang = chunk.get("original_language")
            if chunk_lang not in languages and orig_lang not in languages:
                continue

        results.append((float(score), chunk))

        if len(results) >= top_k:
            break

    return results

## 4. Test Retrieval

In [ ]:
# Test query
question = "What happened at the Battle of Hattin?"

results = retrieve(question, top_k=5)

print(f"Query: '{question}'\n")
print(f"Found {len(results)} relevant chunks:\n")

for i, (score, chunk) in enumerate(results):
    lang = chunk.get("language", "en")
    flag = get_language_flag(lang)
    lang_name = get_language_name(lang)
    is_trans = chunk.get("is_translation", False)
    
    print(f"{'='*60}")
    print(f"Result {i+1}: {chunk['chunk_id']}")
    print(f"Score: {score:.3f} | {flag} {lang_name}" + (" (translated)" if is_trans else ""))
    if chunk.get("author"):
        print(f"Author: {chunk['author']}")
    if chunk.get("title"):
        print(f"Title: {chunk['title']}")
    print(f"\nPreview:\n{chunk['text'][:400]}...\n")

## 5. Context Formatting

Retrieved chunks are formatted with metadata for the LLM.

In [ ]:
def format_context(
    results: list[tuple[float, dict]],
    include_original: bool = False,
    max_chunk_len: int = None,
) -> str:
    """
    Format retrieved chunks into context for the LLM.
    
    Args:
        results: List of (score, chunk) tuples
        include_original: Include original text for translations
        max_chunk_len: Optional max length per chunk
        
    Returns:
        Formatted context string
    """
    parts = []

    for score, chunk in results:
        chunk_id = chunk.get("chunk_id", "unknown")
        lang = chunk.get("language", "en")
        lang_name = get_language_name(lang)
        is_trans = chunk.get("is_translation", False)
        orig_lang = chunk.get("original_language")

        # Build header
        header = f"[{chunk_id}] (score: {score:.3f})"
        if orig_lang and orig_lang != "en":
            orig_name = get_language_name(orig_lang)
            header += f" [Original: {orig_name}, Translated to English]"
        elif lang != "en":
            header += f" [{lang_name}]"

        # Get text
        text = chunk.get("text", "")
        if max_chunk_len and len(text) > max_chunk_len:
            text = text[:max_chunk_len] + "..."

        part = f"{header}\n{text}"

        # Include original if requested
        if include_original and is_trans:
            original = chunk.get("original_text", "")
            if original:
                if max_chunk_len and len(original) > max_chunk_len:
                    original = original[:max_chunk_len] + "..."
                part += f"\n\n[Original {get_language_name(orig_lang)} text:]\n{original}"

        parts.append(part)

    return "\n\n---\n\n".join(parts)

In [ ]:
# Format context from our results
context = format_context(results, max_chunk_len=500)
print("Formatted Context:")
print("=" * 60)
print(context[:2000])
print("...")

## 6. Prompt Templates

Different response modes use different prompt structures.

In [ ]:
SYSTEM_PROMPT = """You are a scholarly historian assistant specializing in the Crusades and the medieval Near East (1095-1291 CE).

You work with PRIMARY SOURCES in multiple languages:
- Latin chronicles (William of Tyre, Fulcher of Chartres, Gesta Francorum)
- Arabic histories (Ibn al-Athir, Usama ibn Munqidh)
- Byzantine Greek sources (Anna Comnena's Alexiad)
- Modern scholarly analysis

CRITICAL RULES:
1. Answer ONLY using the provided CONTEXT - do not use outside knowledge
2. EVERY factual claim must have a citation in format [ChunkID]
3. When citing translated sources, note the original language
4. For disputed events, present multiple perspectives if available
5. Use proper medieval terminology (Outremer, Franks, Saracens where appropriate)
6. Acknowledge when sources conflict or are insufficient

If the answer is not in the CONTEXT, respond:
"The provided sources do not contain sufficient information to answer this question."
"""

MODES = {
    "default": """Provide a scholarly answer with proper citations.
Format: Clear prose with [ChunkID] citations after each claim.
Note the original language of sources when relevant.""",

    "chronology": """Create a detailed timeline of events.
Format:
- [YEAR or c. YEAR] Event description [ChunkID] (Source: Author/Work)

Use approximate dates (c. 1187) when exact dates are unknown.
Cross-reference multiple sources when available.""",

    "dossier": """Write a structured scholarly dossier with these sections:

## Overview
Brief introduction with key dates and significance

## Primary Sources
What do contemporary chronicles say? Cite with [ChunkID] and note languages.

## Key Events
Chronological narrative of major events

## Historical Significance
Why this matters in Crusade history

## Source Notes
List sources used, noting original languages (Latin, Arabic, Greek, etc.)""",

    "comparative": """Compare perspectives across different source traditions.

## Western/Latin Sources
Evidence from Frankish chronicles [ChunkID]

## Eastern/Arabic Sources
Evidence from Muslim historians [ChunkID]

## Byzantine/Greek Sources
Evidence from Eastern Christian sources [ChunkID]

## Analysis
- Points of agreement across traditions
- Significant differences and possible reasons
- Which details are unique to each tradition""",

    "claim_check": """Evaluate the historical claim using available sources.

## Claim
Restate the claim being evaluated

## Primary Evidence
What do medieval sources say? [ChunkID] with source language noted

## Source Agreement
Do sources agree, conflict, or provide different perspectives?

## Verdict
SUPPORTED / PARTIALLY SUPPORTED / NOT SUPPORTED / INSUFFICIENT EVIDENCE

## Confidence
HIGH / MEDIUM / LOW (based on source quality and agreement)""",
}


def build_prompt(mode: str, question: str, context: str) -> str:
    """Build a complete prompt for the LLM."""
    mode_instruction = MODES.get(mode, MODES["default"])
    
    return f"""{SYSTEM_PROMPT}

MODE: {mode.upper()}
INSTRUCTION: {mode_instruction}

QUESTION:
{question}

CONTEXT:
{context}

ANSWER:
"""

## 7. Answer Generation with Gemini

In [ ]:
from google import genai
from google.genai import types

def ask_gemini(prompt: str) -> str:
    """Generate answer using Gemini API."""
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        temperature=0.3,
        max_output_tokens=4096,
        top_p=0.95,
        top_k=64
    )

    resp = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt,
        config=config
    )

    return resp.text or ""

## 8. Complete RAG Pipeline

In [ ]:
def ask_question(
    question: str,
    mode: str = "default",
    top_k: int = 6,
    languages: list[str] = None,
) -> tuple[str, list[tuple[float, dict]]]:
    """
    Complete RAG pipeline: retrieve + generate.
    
    Args:
        question: User's question
        mode: Response mode (default, chronology, dossier, comparative, claim_check)
        top_k: Number of sources to retrieve
        languages: Optional language filter
        
    Returns:
        (answer, results) tuple
    """
    # Step 1: Retrieve relevant chunks
    results = retrieve(question, top_k=top_k, languages=languages)
    
    if not results:
        return "No relevant sources found for this question.", []
    
    # Step 2: Format context
    context = format_context(results)
    
    # Step 3: Build prompt
    prompt = build_prompt(mode, question, context)
    
    # Step 4: Generate answer
    answer = ask_gemini(prompt)
    
    return answer, results

## 9. Interactive Demo

In [ ]:
# Example 1: Default mode
question = "What happened at the Battle of Hattin?"
mode = "default"

print(f"Question: {question}")
print(f"Mode: {mode}")
print("=" * 60)

answer, results = ask_question(question, mode=mode, top_k=6)

print("\nANSWER:")
print(answer)

print("\n" + "=" * 60)
print("SOURCES USED:")
for score, chunk in results:
    lang = chunk.get("language", "en")
    flag = get_language_flag(lang)
    print(f"  {flag} {chunk['chunk_id']} (score: {score:.3f})")

In [ ]:
# Example 2: Comparative mode
question = "How did different sources describe Saladin?"
mode = "comparative"

print(f"Question: {question}")
print(f"Mode: {mode}")
print("=" * 60)

answer, results = ask_question(question, mode=mode, top_k=8)

print("\nANSWER:")
print(answer)

In [ ]:
# Example 3: Chronology mode
question = "What were the major events of the First Crusade?"
mode = "chronology"

print(f"Question: {question}")
print(f"Mode: {mode}")
print("=" * 60)

answer, results = ask_question(question, mode=mode, top_k=8)

print("\nANSWER:")
print(answer)

## 10. Try Your Own Questions

In [ ]:
# Try your own question!
your_question = "Who was Baldwin IV of Jerusalem?"  # <-- Change this
your_mode = "dossier"  # Options: default, chronology, dossier, comparative, claim_check

print(f"Question: {your_question}")
print(f"Mode: {your_mode}")
print("=" * 60)

answer, results = ask_question(your_question, mode=your_mode, top_k=6)

print("\nANSWER:")
print(answer)

print("\n" + "=" * 60)
print("SOURCES:")
for score, chunk in results:
    lang = chunk.get("language", "en")
    flag = get_language_flag(lang)
    print(f"  {flag} {chunk['chunk_id']} (score: {score:.3f})")

## Summary

This notebook demonstrated the complete RAG pipeline:

1. **Retrieval**: Embed the query and search the FAISS index for similar chunks
2. **Context Assembly**: Format chunks with metadata for the LLM
3. **Prompt Engineering**: Build structured prompts for different response modes
4. **Generation**: Use Gemini to generate answers with citations

The system supports multiple response modes:
- **Default**: Scholarly prose with citations
- **Chronology**: Timeline format
- **Dossier**: Structured research report
- **Comparative**: Cross-cultural analysis
- **Claim Check**: Fact verification